# Alternating Least Squares (ALS)

ALS is a type of matrix factorization algorithm used most commonly in recommender systems as a latent factor model. It is a method for collaborative filtering, since it makes recommendations based only on past user-item interactions.

## Problem formulation

The data we are provided with is essentially a ratings matrix, where the number of columns is equal to the number of items $I$ and the number of rows is equal to the number of users $U$.

$$
\begin{bmatrix}
    r_{11}       & r_{12}  & \dots & r_{1I} \\
    r_{21}       & r_{22}  & \dots & r_{2I} \\
     & \dots \\
    r_{U1}       & r_{U2} & \dots & r_{UI}
\end{bmatrix}
$$

This matrix is sparse, and is usually presented in (user, item, rating) triplets. The approach that ALS takes is to represent each user and item in a low-dimensional feature space - each user/item is a vector in some $d$ dimensional space and the affinity (rating) that a user and item have for each other is the dot product of these two vectors (essentially their similarity).

Our goal is then to find user and item factors (d-dimensional vectors) such that we minimize the MSE between the predicted ratings and the true ratings.

$$
\mathcal{L} = \frac{1}{n} \sum_{i, j} \left(r_{ij} - \mathbf{u}_i^T \mathbf{v}_j \right)^2
$$

There are a couple of ways to think of the decomposition of the ratings matrix. First, that each element in the matrix is the dot product of the user latent features and the item latent features. This is a nice way to think about it, since we can think of how a user will like an item as being the sum of the amount that a user likes each latent feature multiplied by the amount of each latent feature that that item possesses (think movie genres). We can also think of the ratings matrix as being approximated by a sum of outer products, where the outer products are the outer products of each latent feature in the user matrix with the same latent feature in the item matrix.

$$
\tilde{R} = \mathbf{d}_1^T \mathbf{d}_1 + \mathbf{d}_2^T \mathbf{d}_2 + \dots + \mathbf{d}_D^T \mathbf{d}_D
$$

Note that SVD accomplishes a similar thing: SVD considers that the $U \times V$ ratings matrix $R$ is the product of two $rank(R)$ matrices. But we know that we can take the D components with the largest singular values and get a low-rank approximation to R that minimizes the Frobenius norm (the MSE) between $R$ and $\tilde{R}$ for rank(D) approximations. SVD can also be thought of as the sum of rank one matrices, where each matrix in the sum corresponds to a single latent feature. SVD takes the latent features that contribute most to this sum, i.e. that have the largest singular values, and drop the rest. The latent features are orthognal to each other.

**TODO** Do SVD and ALS accomplish the same thing? i.e. are the latent features the same? I'm not sure, and I don't think so since there is nothing that says that the latent feature vectors must be orthogonal to one another in ALS. In SVD, we should have that the latent feature vectors in each matrix are orthogonal. (i.e. the columns of the below matrix would be orthogonal). Let's check this with experimentation.

$$
$$
\begin{bmatrix}
    u_{1}\\
    u_{2}\\
    u_{U}\\
\end{bmatrix}
$$
$$

## Solving

It is actually possible to solve this problem via SGD or other optimization method. We can just consider all the user features and item features flattened into a coefficient array and we can take the partial derivatives and make updates using SGD. I'm not too sure about the trade-offs, but you cannot usually be working with that parameter vector all at once since it is $O((U + V)d)$.

Let's consider the case for a single user, user 1. We want to minimize

$$
\mathcal{L} = \frac{1}{n} \sum_{j \in I_1} \left(r_{1j} - \mathbf{u}_1^T \mathbf{v}_j \right)^2\\
\mathcal{L} = \frac{1}{n} \left(\mathbf{r}_1 - V \mathbf{u}_1\right)^T \left(\mathbf{r}_1 - V \mathbf{u}_1\right)
$$

where $I_1$ is the set of all items that user 1 has rated.

This is simply a least squares problem where user 1's latent features are the coefficients, and the matrix of item features is the data matrix, and user 1's true ratings are the labels. We know how to solve this, if we know what the data matrix is (but we don't).

So, if we did know the item features, then we could simply solve $U$ least squares problems independently to find the user factors that minimize the MSE. As we said, though, we don't know our item factors. ALS tackles this problem by starting with some initial item feature matrix, then solving for the best user features. Next, it uses those user features as the data matrix, and solves for the item features. So it alternates between solving a least squares problem for the user factors and then for the item factors, hence the name. We can show that this converges to the optimal solution **TODO**.